In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

BACKGROUND REMOVAL

In [2]:
def remove_background_grabcut(image):
    mask = np.zeros(image.shape[:2], np.uint8)
    bgModel = np.zeros((1, 65), np.float64)
    fgModel = np.zeros((1, 65), np.float64)

    h, w = image.shape[:2]
    rect = (10, 10, w - 20, h - 20)

    cv2.grabCut(image, mask, rect, bgModel, fgModel, 5, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

    img_fg = image * mask2[:, :, np.newaxis]
    return img_fg

HISTOGRAM HSV

In [3]:
def extract_features(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    h, s = hsv[:, :, 0], hsv[:, :, 1]

    hist_h = cv2.calcHist([h], [0], None, [64], [0, 180])
    hist_s = cv2.calcHist([s], [0], None, [32], [0, 256])

    hist = np.concatenate((hist_h.flatten(), hist_s.flatten()))
    hist = hist / np.sum(hist)

    return hist

LOAD DATASET

In [4]:
def load_dataset(path):
    X, y = [], []

    for label, class_name in enumerate(["ORGANIK", "ANORGANIK"]):
        folder = os.path.join(path, class_name)

        print("Memproses folder:", folder)

        for file in os.listdir(folder):
            img_path = os.path.join(folder, file)

            img = cv2.imread(img_path)
            if img is None:
                continue

            img = cv2.resize(img, (256, 256))

            # Background removal
            img_no_bg = remove_background_grabcut(img)

            # Smoothing
            img_blur = cv2.GaussianBlur(img_no_bg, (5, 5), 0)

            # Extract features
            features = extract_features(img_blur)

            X.append(features)
            y.append(label)

    return np.array(X), np.array(y)

OUTPUT (gambar asli -> no backgroung -> smooth)

In [6]:
def show_processing_steps(image):
    img = cv2.resize(image, (256, 256))
    img_no_bg = remove_background_grabcut(img)
    img_blur = cv2.GaussianBlur(img_no_bg, (5, 5), 0)

    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.title("Gambar Asli")
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.title("Tanpa Background")
    plt.imshow(cv2.cvtColor(img_no_bg, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.title("Setelah Smoothing")
    plt.imshow(cv2.cvtColor(img_blur, cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.show()

In [8]:
test_img = cv2.imread("./O_1.jpeg")
show_processing_steps(test_img)

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
